## Introdução - Responsible AI

### Prof. Ahirton Lopes (profahirton.lopes@fiap.com.br)

Este notebook explora conceitos de **IA Responsável** na prática, utilizando uma abordagem de fairness para avaliar viés em modelos de previsão de reincidência criminal. Aqui, analisamos a base de dados COMPAS e aplicamos métricas para medir e mitigar viés algorítmico.

Vamos começar carregando os pacotes necessários!

In [36]:
!pip install aif360
!pip install aif360[Reductions]
!pip install aif360[inFairness]

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing

## Carregamento dos Dados

Neste bloco, carregamos os dados da base COMPAS, que contém informações sobre indivíduos e suas chances de reincidência criminal. Esta base já foi analisada em estudos sobre viés algorítmico.

In [38]:
# Carregar os dados COMPAS
df = pd.read_csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")

## Pré-processamento dos Dados

Aqui realizamos o tratamento dos dados, removendo colunas irrelevantes e transformando variáveis categóricas em formato adequado para o modelo. Também identificamos o atributo protegido ('race') para a análise de fairness.

In [39]:

# Selecionar colunas relevantes
cols = ["sex", "age", "race", "priors_count", "c_charge_degree", "two_year_recid"]
data = df[cols].copy()

# Filtrar apenas as raças mais presentes na base
data = data[data['race'].isin(["African-American", "Caucasian"])]

# Transformar variáveis categóricas
data['race'] = data['race'].map({"African-American": 1, "Caucasian": 0})
data['sex'] = data['sex'].map({"Male": 1, "Female": 0})
data['c_charge_degree'] = data['c_charge_degree'].map({"F": 1, "M": 0})

In [40]:
# Separar features e target
X = data.drop(columns=["two_year_recid"])
y = data["two_year_recid"]

In [41]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## Treinamento do Modelo

Utilizamos um modelo de Machine Learning para prever a reincidência criminal. O objetivo é avaliar como o modelo se comporta em relação a diferentes grupos da população.

In [42]:
# Treinar um modelo simples
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [43]:
# Fazer previsões
y_pred = model.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("Relatório de classificação:\n", classification_report(y_test, y_pred))

Acurácia: 0.6200542005420054
Matriz de confusão:
 [[645 340]
 [361 499]]
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.64      0.65      0.65       985
           1       0.59      0.58      0.59       860

    accuracy                           0.62      1845
   macro avg       0.62      0.62      0.62      1845
weighted avg       0.62      0.62      0.62      1845



## Avaliação de Fairness

Aqui aplicamos métricas de IA Responsável para medir possíveis vieses do modelo. Usamos métricas como:
- **Disparate Impact**: Mede a diferença na taxa de previsões favoráveis entre grupos protegidos e não protegidos.
- **Equal Opportunity Difference**: Diferença na taxa de verdadeiros positivos entre grupos.
- **False Positive Rate Difference**: Diferença na taxa de falsos positivos entre grupos protegidos e não protegidos.

In [44]:
from aif360.metrics import ClassificationMetric

# Criar um dataset binário para fairness analysis
dataset = StandardDataset(pd.concat([X_test, y_test], axis=1),
                          label_name="two_year_recid",
                          protected_attribute_names=["race"],
                          privileged_classes=[[0]],  # Caucasian como classe privilegiada
                          favorable_classes=[0])  # Não reincidir como favorável

# Criar dataset com previsões
dataset_pred = dataset.copy()
dataset_pred.labels = y_pred.reshape(-1, 1)  # Garantir que as previsões tenham o formato correto

# Criar uma métrica de imparcialidade

metric = BinaryLabelDatasetMetric(dataset, privileged_groups=[{"race": 0}], unprivileged_groups=[{"race": 1}])

# Criar objeto de métricas de classificação
metric_classification = ClassificationMetric(dataset, dataset_pred,
                                             privileged_groups=[{"race": 0}],
                                             unprivileged_groups=[{"race": 1}])

# Calcular métricas de fairness
print("Disparate Impact:", metric.disparate_impact())  # Já calculado antes
print("Equal Opportunity Difference:", metric_classification.equal_opportunity_difference())  # Diferença na taxa de verdadeiros positivos
print("False Positive Rate Difference:", metric_classification.false_positive_rate_difference())  # Diferença na taxa de falsos positivos

Disparate Impact: 0.8001111668287849
Equal Opportunity Difference: -0.14181967662373252
False Positive Rate Difference: -0.17604781768604294


## Mitigação de Viés

Se um viés for identificado, aplicamos técnicas de fairness para reduzir esse problema. Neste caso, utilizamos o método **Reweighing**, que ajusta os pesos das amostras para tornar as previsões mais justas.

In [45]:
# Mitigação de viés usando reweighting
reweigh = Reweighing(privileged_groups=[{"race": 0}], unprivileged_groups=[{"race": 1}])
reweigh.fit(dataset)
reweighed_dataset = reweigh.transform(dataset)

metric_after = BinaryLabelDatasetMetric(reweighed_dataset, privileged_groups=[{"race": 0}], unprivileged_groups=[{"race": 1}])
print("Disparate Impact após reweighting:", metric_after.disparate_impact())

Disparate Impact após reweighting: 1.0000000000000002


## Conclusão

Finalizamos a análise avaliando o impacto das técnicas de mitigação e destacando a importância de modelos de IA que sejam **transparentes e justos**.